# ML Jour 2 – Cas Olist

Pour les 2 premiers jours de ML, nous utiliserons à nouveau les données Olist. Le dataset est disponible sur Notion dans ML Day 1, sur le serveur DBeaver et sur [Kaggle](https://www.kaggle.com/olistbr/brazilian-ecommerce).

Le but de ce use case est de monter en compétence sur les différentes techniques et modèles que nous verrons en cours, et d'apprendre à utiliser Dataiku pour mettre en place un workflow de machine learning.

**Objectif** : Prédire la customer satisfaction après réception d'une commande. Notre objectif sera donc de prédire la note de la review associée à une commande. Notre variable cible `y` est donc la variable *review_score*.

**Jour 1** : Pour commencer, on va se concentrer sur l'EDA et sur la phase de pré-processing des données, de manère à produire un dataset propre utilisant un subset de variables du dataset.

**Jour 2** : Ensuite, on utilisera ce dataset pour effectuer une régression linéaire, puis une régression logistique.

**Jour 3** : Enfin, nous mettrons à profit ce que nous avons vu sur la plateforme de ML no-code Dataiku, en répliquant le pipeline entier du pré-processing jusqu'au modèle de régression linéaire.

Pour les 3 jours, l'objectif est de **comprendre le fonctionnement d'un pipeline de ML sur Dataiku et sur Python**. Le plus important n'est pas de connaître par cœur le code mais de comprendre qu'un pipeline de ML est très standardisé quel que soit le type de problème (régression, classification).

## Data loading

Nous allons utiliser le dataset nettoyé que nous avons construit hier.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option("max_columns", 100)

In [ ]:
df_olist = pd.read_csv("olist_dataset/olist_cleaned.csv")

In [ ]:
df_olist.head(3)

## Etape 4 : normalisation et encoding des features catégorielles


### Normalisation

Normalisez les variables numériques, à l'exception de la variable cible, à l'aide du `StandardScaler`.

Recréez un dataframe final, qui contient toutes les variables numériques normalisées et la variable cible.

In [ ]:
# import and instantiate StandardScaler object
from sklearn.preprocessing import StandardScaler

standard_scal = StandardScaler()

In [ ]:

# select only numeric variables

num_cols = df_olist.select_dtypes(include="number").drop(columns='review_score').columns
df_olist_num = df_olist.copy()
df_olist_num = df_olist_num[num_cols]
df_olist[num_cols] = standard_scal.fit_transform(df_olist_num.values)

df_olist.head(3)

### Encoder les variables catégorielles
Isolez les 3 variables catégorielles "payment_type", "order_status", "product_category_name" et affichez des `value_counts`.

Utilisez la méthode du one-hot encoding sur ces 3 variables. Pour les catégories de produits, vous pouvez regrouper les petites catégories sous le nom "other".

Il est également possible ici d'encoder les informations géographiques. Créez 2 nouvelles colonnes, qui montrent respectivement si le vendeur et l'acheteur sont dans le même état et la même ville.

In [ ]:
categorical_variables = ["payment_type", "order_status", "product_category_name"]

In [ ]:
df_olist.head(3)

In [ ]:
df_olist["product_category_name"].nunique()

Dans notre cas, il existe 73 modalités différentes pour le nom de catégorie. C'est très élevé et cela risque de perturber le modèle de ML ! Nous allons donc rassembler les modalités en ne gardant que les 20 premières (c'est un choix arbitraire) tout en rassemblant les autres dans une catégorie "autres".

In [ ]:
other_categories = df_olist["product_category_name"].value_counts().index[20:]

In [ ]:
def category_group(x):
    if x in other_categories:
        return "other"
    else:
        return x

In [ ]:
df_olist["product_category_name"] = df_olist["product_category_name"].apply(lambda x: category_group(x))

In [ ]:
# vérification de la nouvelle catégorie "other"
df_olist["product_category_name"].value_counts()

In [ ]:
# encoding des catégories de "product_category_name"
ohe_product_categ = pd.get_dummies(df_olist["product_category_name"], prefix="categ")
df_olist = pd.concat([df_olist, ohe_product_categ], axis=1)

In [ ]:
# encoding des catégories de "order_status"
ohe_order_status = pd.get_dummies(df_olist["order_status"], prefix="order_status")
df_olist = pd.concat([df_olist, ohe_order_status], axis=1)

In [ ]:
# encoding des catégories de "payment_type"
ohe_payment_type = pd.get_dummies(df_olist["payment_type"], prefix="payment_type")
df_olist = pd.concat([df_olist, ohe_payment_type], axis=1)
df_olist

Cependant, il est cette fois nécessaire de supprimer toutes les colonnes de date et les colonnes catégoriques qui n'ont pas été encodées, car elles ne sont pas interprétables par le modèle, qui va crasher.

In [ ]:
cols_to_drop = ['product_category_name', 'order_status', 'payment_type'] 

In [ ]:
df_olist = df_olist.drop(columns=cols_to_drop)

In [ ]:
df_olist = df_olist.dropna()

In [ ]:
df_olist.head(3)

## Etape 4 - Fit

### Création des 4 datasets utilisés par le modèle

1) Séparer les features (`X`) de la target (`y`).

2) Réaliser un train-test split qui va permettre de construire nos 4 datasets de fin : X train, X test, y train et y test.

Réaliser un train-test split avec 75% des données en train. Ajouter `random_state=42` en argument de la fonction `train_test_split` afin que l'on ait des résultats comparables.

<blockquote>Comment un train-test split fonctionne-t-il ? A partir d'un dataset X (nos variables de prédiction) et d'une series y (notre variable à prédire), on va prendre un échantillon de lignes de X et de y (avec les mêmes index) pour faire le train dataset, et le reste sera le test dataset. Par exemple, si l'on prend un dataset de 10 lignes avec 80% des lignes dans le train, on choisit 8 index au hasard (ex: 0, 1, 2, 4, 5, 7, 8, 9) du dataset. Ensuite, les lignes correspondantes de X forment X train tandis que les lignes restantes forment X test. De même, les lignes correspondantes de y forment y train tandis que les lignes restantes forment y test.
</blockquote>

![Train test split](train_test_split_2.png)

3 remarques :
- Il faut absolument conserver les mêmes index entre X et y, sinon les features qui servent à prédire s'entraîneront sur les mauvais y !
- Il est possible de faire le découpage des 4 datasets en faisant d'abord le train test split puis en séparant ensuite le train et le test dataset entre X et y. Cela ne change rien, les 2 manières de faire étant identiques tant qu'elles amènent bien à X train, y train, X test et y test.
- En pratique, nous utiliserons la fonction train_test_split de scikit-learn qui s'occupe automatiquement de splitter X et y.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# on exclut bien sûr la colonne review_score des features

X = df_olist.drop("review_score", axis=1)
y = df_olist["review_score"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
X_train.describe()

Nous pouvons maintenant construire le modèle de régression linéaire. L'objet s'importe facilement depuis la librairie scikit-learn. Nous allons également importer la métrique de r2 qui servira à évaluer le modèle.

In [ ]:
# on importe les sous-modules de sklearn dont on a besoin

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [ ]:
# on instancie un objet LinearRegression, puis on le "fit"
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

In [ ]:
# on utilise cet objet pour prédire les valeurs sur le dataset
y_pred = lin_reg.predict(X)

In [ ]:
# on évalue les prédictions avec le R2
score = r2_score(y, y_pred)
print(f"Le R2 est de {score:.1%}!")

### Etape 5 - Evaluation

Nous allons analyser le résultat de la régression linéaire :

- Analyse des valeurs des coefficients: afficher un graphique de feature importance en utilisant la fonction vue dans le live-coding. Ces résultats vous semblent-ils cohérents?

- Significativité des coefficients (t-statistic and p-value), intervalle de confiance des coefficients: informations disponibles dans le résumé `statsmodel` si vous utilisez ce package.

In [ ]:
# on affiche les coefficients avec les features associées

for feat, coef in zip(X.columns, lin_reg.coef_):
    print(feat, f"{coef:.4f}")

In [ ]:
def feature_importance(model, X_train):
    """
    Plots a feature importance graph for regressions (linear, logistic, regularizations...)
    or random forest models.
    
    Args:
        model: trained model
        X_train: the training dataframe, to extract variable names
    """
    
    try:
        try:
            importance = model.coef_[0]
            test_error = importance[0]
        except:
            importance = model.coef_

        importances = []
        for i, v in enumerate(importance):
            importances.append((X_train.columns[i], v))
        importances.sort(key=lambda tup: abs(tup[1]), reverse=True)
    
        feature_names = [x[0] for x in importances]
        importances = [x[1] for x in importances]
        
    
    except:
        try:
            ordering = np.argsort(model.feature_importances_)[::-1]#[:50]
            importances = model.feature_importances_[ordering]

            X_columns = X_train.columns
            feature_names = X_columns[ordering]
        
        except:
            print('The function can only plot feature importance for regression or RF models.')
        
    ticks = np.arange(len(importances))
    fig, ax = plt.subplots(figsize=(16,5))
    ax = sns.barplot(y=importances, x=ticks, palette=sns.diverging_palette(150, 10, center="dark", n=len(importances)))
    plt.xticks(ticks, feature_names, rotation=90)
    plt.show()

In [ ]:
feature_importance(lin_reg, X)

# Régression logistique

Pour vous montrer que les pipelines suivent la même structure même dans le cas de deux problèmes différents, nous allons maintenant construire une régression logistique.

Dans ce cas précis, comme la valeur à prédire est discrète, nous pouvons au choix effectuer une régression ou une classification. Si la valeur à prédire avait été continue comme un prix par exemple, nous n'aurions pu utiliser que la régression.

Essayons donc de refaire le même procédé avec une régression logistique. Le préprocessing est identique, importons donc juste le nouvel objet de régression logistique en conservant les mêmes datasets.

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
print(y_train)

In [ ]:
# on instancie un objet LinearRegression, puis on le "fit"
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

In [ ]:
# on utilise cet objet pour prédire les valeurs sur le dataset
y_pred = lin_reg.predict(X)

In [ ]:
# on évalue les prédictions avec le R2
score = r2_score(y, y_pred)
print(f"Le R2 est de {score:.1%}!")

In [ ]:
feature_importance(lin_reg, X)